In [2]:
from langchain_openai import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore

from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

from dotenv import load_dotenv
load_dotenv(".env")

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")



splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.pdf")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

chached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = Chroma.from_documents(docs, chached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    ("human", "{question}")
])

# RunnablePassthrough 는 입력값이 말 그대로 통과하게 해 주는 거다.
# 입력값이 저기에 전달되는거다.

# 결국 [DOC] 랑 QUSTION 이  프롬프트로 들어가는거다.
chain = {"context": retriver, "question": RunnablePassthrough(), "extra": RunnablePassthrough()} | prompt | llm

chain.invoke("winston은 어디에 살어?")




AIMessage(content='Winston은 Victory Mansions에 살고 있습니다.')